In [1]:
import os
import shutil

import openpyxl
import pandas as pd

# 作成ライブラリの読み込み
from lib.lib_directory import *
from lib.lib_shape import *
from lib.lib_shape_process import *
import lib.lib_init as var

### 初期設定(実行前に確認・変更を行ってください)

In [2]:
#########################################
# 初期設定(実行前に確認・変更を行ってください) #
#########################################

# データフローを記載するExcelファイル名
filename = './sample.xlsx'

# データフローを記載するシート名
sheetname = 'データフロー図'

# workflowのディレクトリ構造を出力ディレクトリ
output_dir = './output/'

# 各種設定のtemplateを格納しているディレクトリ
template_dir = './template/'

# workflowのプロジェクト名
project_name = 'sample_project01'

### 以降、プログラム処理の部分のため必要がない場合は修正を行わない

In [3]:
# workflowディレクトリの生成
base_dir = output_dir + project_name

# template_management.csvを取り込み
df = pd.read_csv(template_dir + 'template_management.csv')

# Excelデータの読み込み(フロー記述のExcel)
wb = openpyxl.load_workbook(filename, data_only=True)

In [8]:
# 設定内容一覧のセクション開始文字列
list_section = ['設定内容一覧', 'カラム設定', '特記事項']
start_col = 2
start_row = 2

for sheet in wb.sheetnames:
  if sheet == sheetname:
    continue
  book = wb[sheet]

  list_section_point = []
  for i in range (start_row, book.max_row):
    if book.cell(i, start_col).value in list_section:
      #print(book.cell(i, start_col).value + ':' + str(i))
      list_section_point.append(i)
  list_section_point.sort()
  
  for i in range (0, len(list_section_point) - 1):
    skiprows = list_section_point[i]
    skipfooter = book.max_row - (list_section_point[i + 1] - 2)
    df = pd.read_excel(filename, sheet_name=sheet, skiprows=skiprows, skipfooter=skipfooter)
  break  

In [9]:
df

,Unnamed: 0,No.,物理名,論理名,型,フォーマット,タイムゾーン,説明,Unnamed: 8,Unnamed: 9
0,NaN,1,test01,テスト1,boolean,NaN,NaN,NaN,NaN,NaN
1,NaN,2,test02,テスト2,long,NaN,NaN,NaN,NaN,NaN
2,NaN,3,test03,テスト3,timestamp,%Y-%m-%d %H:%M:%S,Asia/Tokyo,NaN,NaN,NaN
3,NaN,4,test04,テスト4,double,NaN,NaN,NaN,NaN,NaN
4,NaN,5,test05,テスト5,string,NaN,NaN,NaN,NaN,NaN


In [22]:
33 - 18

15

In [ ]:
# 図形情報の辞書化
dict_org = {}
dict_org = get_shape_dict(sheet)

# 図形付加情報,X軸下端,Y軸右端を追加
dict_org = add_shape_info(dict_org)

# 図形と線を分離
dict_shape = {}
dict_line = {}
dict_shape, dict_line = split_shape_line_dict(dict_org)

# 図形のin_point/out_pointを付与
dict_shape = add_shape_info_point(dict_shape)

# 図形のin_line_no/out_line_noを付与
dict_shape = add_shape_info_lineno(dict_shape, dict_line)

# 図形の処理を全て終わるまでループしてworkflowファイル群を生成
main_shape_process(dict_shape, fp, template_dir, base_dir, df, wb_tmp, wb)

# mainとなるdigのファイルをクローズ
fp.close()